## Frida 教程(一)
　　参考：[Frida 官方文档](https://frida.re/docs/javascript-api/#java)<br>
　　　　　[Frida操作手册](https://github.com/hookmaster/frida-all-in-one)<br>
　　　　　[win10系统下-frida环境安装](https://www.52pojie.cn/thread-1100931-1-1.html)<br>
　　　　　[Android模拟器安装arm支持](https://my.oschina.net/sannychan/blog/647564)<br>
　　　　　[Frida&FART Git教程](https://github.com/r0ysue/AndroidSecurityStudy)<br>
　　　　　[Frida 脚本系列1-4[《 FRIDA系列文章》](https://github.com/r0ysue/AndroidSecurityStudy)](https://www.anquanke.com/member/131652)<br>
     
### Frida Hook调试APP环境安装
#### 1. 安装frida-tools
　　这里给出的是python的安装指令: <br>
>pip install frida-tools<br>

　　查看CPU内核(根据内核版本去下载相应frida-server)：<br>
> su cat /proc/cpuinfo　　　# linux查看内核
#### 2. 安装adb
　　在终端中执行指令:
>brew cask install android-platform-tools<br>

　　windows中则推荐到http://adbshell.com/downloads下载ADB Kits，解压后得到adb.exe、AdbWinApi.dll、AdbWinUsbApi.dll三个文件，然后将这三个文件拷贝到C:\Windows\System32\目录下，保险起见，同时拷贝这三个文件到C盘中的Windows/System/目录下 及Windows/SysWOW64/；如果不拷贝到System/目录下有可能执行adb时会出错："找不到AdbWinApi.dll文件" 如果不拷贝到Windows/SysWOW64/有可能执行adb时会出错"CreateProcess failure. error 2"<br>
　　Windows安装方法参考: windows安装adb方法及问题解决<br>
　　然后测试是否安装成功: adb devices, 出现以下形式内容则说明安装成功<br>
>List of devices attached<br>
e312073d    device
#### 3. 安装frida-server
　　到github/frida下载frida-server, 选择自己所要操作的对应版本的server就可以了, 安卓设备注意对应自己设备的架构(例如arm, arm64, x86等等, 安装了xposed可以在xposed中看到自己设备对应的架构)<br>
　　**注意:** 一定要把frida-server版本和上面PC端安装的frida版本一致，不然运行报错的。<br>
　　查看设备架构：<br>
>cat /proc/cpuinfo　　　# 查看CPU信息（参考：[查看Android设备的CPU架构信息
](https://www.jianshu.com/p/d65885e1664c)）

　　下载完成后将文件名改为frida-server(改成自己想要的名称都可以, 不用解压), 然后放到指定目录下, 一般安卓手机放在/data/local/tmp/目录下<br>
　　执行frida-ps，验证是否安装成功：<br>
>frida-ps -U 　　　　　# PC端执行，如果出现Android的进程，则代表安装成功。

　　注意：入口看到下面这样“frida-ps 不是内部或外部命令” ，需要再安装frida-tools<br>
#### 4. 启动frida-server
　　首先连接安装有frida的设备(需打开USB调试), 终端运行指令: <br>
> adb root　　　　# 以root运行adb<br>

　　查看设备是否连接:<br> 
> adb devices<br>

　　进入adb shell: <br>
> adb shell<br>

　　在adb shell中执行frida-server运行指令(首次执行前最好先修改frida-server文件的权限):<br>
> chmod 755 /data/local/tmp/frida-server<br>
/data/local/tmp/frida-server & 　　　　# 后面加上&使其在后台执行, 目录即为frida-server的存放目录。<br>

　　如果需要调试的，启动frida-server后，一定要设置端口转发到pc端<br>
>adb forward tcp:27042 tcp:27042<br>
adb forward tcp:27043 tcp:27043<br>

　　运行完成后可通过exit;退出adb shell<br>
  
#### 5. 调试安卓app
　　frida hook调试app首先需要该app的源码, 找到自己想要调试的函数的位置。一切准备就绪后，就可以开始调试安卓程序了，参考以下步骤<br>
　　在运行有frida-server的设备上安装需要调试的app<br>
　　运行app然后在终端里运行指令: adb shell dumpsys activity top即可查看该app的包名, 这里假设我们找到的是com.xxx.xxx，然后使用python编写hook脚本：
>\# -*- coding:utf-8 -*-<br>
import frida, sys<br>
<br>
\# frida -U --no-pause -f "com.xxx.xxx"<br>
package_name = "com.xxx.xxx" # 该app的包名<br>
def on_message(message, data):<br>
 if message\['type'\] == 'send':<br>
     print("\[\*\] {0}".format(message\['payload'\]))<br>
 else:<br>
     print(message)<br>
<br>
\# hook脚本<br>
jscode = """<br>
Java.perform(function () {<br>
 console.log("start hook")<br>
 var userLogin = Java.use("com.a.b.c"); //com.a.b.c中c是需要调试函数所在的类名, com.a.b是c类所在的包名<br>
 send("userLogin: " + userLogin);<br>
 userLogin.Encrypt.overload("java.lang.String", "java.lang.String").implementation = function(a1, a2){<br>
     console.log(a1);<br>
     var result = this.Encrypt(a1, a2);<br>
     console.log(result);<br>
     return result;<br>
 }// Encrypt是我们想要hook的函数，"java.lang.String"则为相应位置的参数类型占位, a1,a2即为该函数的两个参数<br>
});<br>
"""<br>
<br>
device = frida.get_usb_device()<br>
print("\[\*\] 找到设备")<br>
\# pid = device.spawn(\[package_name\])<br>
\# print(pid)<br>
process = device.attach(package_name)<br>
print("\[\*\] 找到进程")<br>
\# device.resume(pid)<br>
script = process.create_script(jscode)<br>
script.on('message', on_message)<br>
print('\[\*\] Running CTF')<br>
script.load()<br>
sys.stdin.read()<br>
process.detach()<br>

　　由于进程过多，可能导致上述代码不能成功执行hook功能，此时可通过在终端中执行frida -U --no-pause -f "com.xxx.xxx"来打开app(需将设备中待调试的app退出)， 然后再将hook脚本代码贴到终端中即可实现hook功能<br>